In [6]:
import math as math
import numpy as np
from Py6S import *
import time
#from numba import cuda
from joblib import Parallel, delayed
from scipy.interpolate import griddata,interpn

sza = np.linspace(0,80,17)
vza = np.linspace(0,80,17)
water = np.linspace(0,7,8)
ozone = np.linspace(0.2,0.4,5)
AL = np.linspace(0,8,5)
AOT = np.array([0.01,0.05,0.1,0.15,0.2,0.3,0.4,0.6,0.8,1.0,1.5,2.0]) 
raa = np.linspace(0,180,19)
Aeropro = np.array([1,2])

def ac_band1(In_Aero,In_ozone,In_AOT,In_AL,In_sza,In_vza,In_raa):
       
    wl_band = "/data/Projects/sixs_band1.csv"
    band = np.loadtxt(wl_band,delimiter=",")
    
    s = SixS()
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(3,ozone[In_ozone])
    s.aero_profile = AeroProfile.PredefinedType(Aeropro[In_Aero])
    s.aot550 = AOT[In_AOT]
    s.wavelength = Wavelength(band[0,0],band[band.shape[0]-1,0],band[:,1])
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(AL[In_AL])
    s.geometry = Geometry.User()
    s.geometry.solar_z = sza[In_sza]
    s.geometry.solar_a = raa[In_raa]
    s.geometry.view_z = vza[In_vza]
    s.geometry.view_a = 0
    
    s.atmos_corr = AtmosCorr.AtmosCorrLambertianFromReflectance(0.2)
    s.run()
    
    
    f1 = 1/(s.outputs.transmittance_total_scattering.total*s.outputs.transmittance_global_gas.total)
    return(f1, s.outputs.coef_xb, s.outputs.coef_xc)
    del s



In [7]:
start=time.time()
AC_output = Parallel(n_jobs=32)(delayed(ac_band1)(In_Aero,In_ozone,In_AOT,In_AL,In_sza,In_vza,In_raa) \
                                for In_Aero in range (len(Aeropro)) \
                                for In_ozone in range (len(ozone)) \
                                for In_AOT in range (len(AOT))\
                                for In_AL in range (len(AL))\
                                for In_sza in range (len(sza))\
                                for In_vza in range (len(vza))\
                                for In_raa in range (len(raa)))
end=time.time()

T= end-start
print('time: {:.1f} secs, {:.1f} mins,{:.1f} hours'.format(T,T/60,T/3600) )



time: 247928.1 secs, 4132.1 mins,68.9 hours


In [9]:
X=np.array(AC_output)
X1=X[:,0]
X2=X[:,1]
X3=X[:,2]
outfile1 = "01_band1.csv"
outfile2 = "02_band1.csv"
outfile3 = "03_band1.csv"
np.savetxt(outfile1,X1,delimiter=',')
np.savetxt(outfile2,X2,delimiter=',')
np.savetxt(outfile3,X3,delimiter=',')

In [7]:
outfile1 = "Continental_01_band3.csv"
outfile2 = "Continental_02_band3.csv"
outfile3 = "Continental_03_band3.csv"
X1 = np.loadtxt(outfile1,delimiter=",")
X2 = np.loadtxt(outfile2,delimiter=",")
X3 = np.loadtxt(outfile3,delimiter=",")

In [8]:
X1_reshape=X1.reshape(8,5,12,17,17,19)
X2_reshape=X2.reshape(8,5,12,17,17,19)
X3_reshape=X3.reshape(8,5,12,17,17,19)

In [9]:
xi,yi=np.ogrid[0:80:161j, 0:80:161j]

In [7]:
def get_point():
    A=[]
    for i in range(len(vza)):
        for j in range(len(sza)):
            A.append(vza[i])
            A.append(sza[j])
    point=np.array(A).reshape(17*17,2)
    return point

In [8]:
def griddata_inter(X1,X2,X3,a,b,c,d):
    X1_new_inter=[]
    X2_new_inter=[]
    X3_new_inter=[]
    
    X1_inter=X1[a,b,c,:,:,d].reshape(17*17,1)
    X2_inter=X2[a,b,c,:,:,d].reshape(17*17,1)
    X3_inter=X3[a,b,c,:,:,d].reshape(17*17,1)
    
    X1_new = griddata(point, X1_inter, (xi, yi), method='nearest')
    X2_new = griddata(point, X2_inter, (xi, yi), method='nearest')
    X3_new = griddata(point, X3_inter, (xi, yi), method='nearest')
                
    X1_new_inter.append(X1_new)
    X2_new_inter.append(X2_new)
    X3_new_inter.append(X3_new)
    
    return X1_new_inter,X2_new_inter,X3_new_inter

In [654]:
point=get_point()

In [606]:
output = Parallel(n_jobs=-1)(delayed(griddata_inter)(X1_reshape,X2_reshape,X3_reshape,a,b,c,d)\
                             for a in range(len(water)) \
                             for b in range(len(ozone)) \
                             for c in range(len(AOT)) \
                             for d in range(len(raa)))

In [607]:
X1_new_inter_reshape=np.array(output)[:,0].reshape(8,5,12,19,161,161)
X2_new_inter_reshape=np.array(output)[:,1].reshape(8,5,12,19,161,161)
X3_new_inter_reshape=np.array(output)[:,2].reshape(8,5,12,19,161,161)

In [574]:
sza_new = np.linspace(0,80,161)
vza_new = np.linspace(0,80,161)

In [638]:
points=(water,ozone,AOT,sza,vza,raa)

In [645]:
x_v=np.array([0.35,0.245,1.5,67.9,41.4,17])

In [646]:
res_1 = interpn(
	points,
	X1_reshape,x_v,
	fill_value=True,
	bounds_error=False,
    method='linear')

In [647]:
res_2 = interpn(
	points,
	X2_reshape,x_v,
	fill_value=True,
	bounds_error=False,
    method='linear')

In [648]:
res_3 = interpn(
	points,
	X3_reshape,x_v,
	fill_value=True,
	bounds_error=False,
    method='linear')

In [649]:
wl_band = "/data/Projects/sixs_band3.csv"
band = np.loadtxt(wl_band,delimiter=",")
    
q = SixS()
q.atmos_profile = AtmosProfile.UserWaterAndOzone(0.35,0.245)
q.aero_profile = AeroProfile.PredefinedType(2)
q.aot550 = 1.5
q.wavelength = Wavelength(band[0,0],band[band.shape[0]-1,0],band[:,1])
q.altitudes.set_sensor_satellite_level()
q.altitudes.set_target_custom_altitude(0)
q.geometry = Geometry.User()
q.geometry.solar_z = 67.9
q.geometry.solar_a = 17
q.geometry.view_z = 41.4
q.geometry.view_a = 0
    
q.atmos_corr = AtmosCorr.AtmosCorrLambertianFromReflectance(0.2)
q.run()
    
    
m1 = 1/(q.outputs.transmittance_total_scattering.total*q.outputs.transmittance_global_gas.total)


In [650]:
Ref=0.4
y = res_1*Ref-res_2
TOC_inter=y/(1+res_3*y)
y1 = m1*Ref-q.outputs.coef_xb
TOC_6s=y1/(1+q.outputs.coef_xc*y1)

In [651]:
TOC_inter-TOC_6s

array([0.00529828])